In [17]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, accuracy_score
from sklearn.preprocessing import LabelEncoder

df = pd.read_csv("veri/dataset_with_anomalies.csv")


df.fillna(df.mean(), inplace=True)

# Kategorik sütunlar için en yaygın değeri kullanarak doldurma
categorical_cols = df.select_dtypes(include=['object']).columns
for col in categorical_cols:
    df[col].fillna(df[col].mode()[0], inplace=True)

# Kategorik verileri sayısal hale getirme (Label Encoding)
label_encoder = LabelEncoder()
for col in categorical_cols:
    df[col] = label_encoder.fit_transform(df[col])

# Hedef değişkeni belirleme (anomaly sütunu)
# Eğer anomaly sütunu 0 ve 1 değerleri taşıyorsa, onu hedef olarak alabiliriz
X = df.drop('anomaly', axis=1)  # Özellikler (X)
y = df['anomaly']  # Hedef değişken (y)

# Eğitim ve test setine ayırma
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Random Forest modelini kurma ve eğitme
rf_model = RandomForestClassifier(n_estimators=100, random_state=42)
rf_model.fit(X_train, y_train)

# Test seti üzerinde tahmin yapma
y_pred = rf_model.predict(X_test)

# Model performansını değerlendirme
print("Model Doğruluğu:", accuracy_score(y_test, y_pred))
print("\nClassification Report:\n", classification_report(y_test, y_pred))

# Feature Importance (Özelliklerin Önem Dereceleri)
feature_importances = pd.DataFrame(rf_model.feature_importances_,
                                   index=X.columns,
                                   columns=['Importance']).sort_values('Importance', ascending=False)

print("\nÖzelliklerin Önem Dereceleri:")
print(feature_importances)


Model Doğruluğu: 0.9738066270664854

Classification Report:
               precision    recall  f1-score   support

           0       0.87      0.58      0.70       729
           1       0.98      1.00      0.99     13244

    accuracy                           0.97     13973
   macro avg       0.93      0.79      0.84     13973
weighted avg       0.97      0.97      0.97     13973


Özelliklerin Önem Dereceleri:
                      Importance
minimum_nights          0.133102
availability_30         0.104078
availability_365        0.081758
review_scores_rating    0.070260
property_type           0.068297
longitude               0.064910
latitude                0.063700
number_of_reviews       0.059370
host_name               0.056897
host_location           0.056801
name                    0.051174
price                   0.050657
host_since              0.044226
host_acceptance_rate    0.036550
room_type               0.035806
maximum_nights          0.022415
